In [2]:
import pandas as pd

df1 = pd.read_parquet('/kaggle/input/avito-ml-cup-default-dataset/test_part_0001.snappy.parquet')
df2 = pd.read_parquet('/kaggle/input/avito-ml-cup-default-dataset/test_part_0002.snappy.parquet')
df3 = pd.read_parquet('/kaggle/input/avito-ml-cup-default-dataset/train_part_0001.snappy.parquet')
df4 = pd.read_parquet('/kaggle/input/avito-ml-cup-default-dataset/train_part_0002.snappy.parquet')
df5 = pd.read_parquet('/kaggle/input/avito-ml-cup-default-dataset/train_part_0003.snappy.parquet')
df6 = pd.read_parquet('/kaggle/input/avito-ml-cup-default-dataset/train_part_0004.snappy.parquet')
print('data loaded')


data loaded


In [3]:
df3['base_json_params'][0]

'{"178": 758, "179": 768, "2756": 19916, "2827": 20032, "110064": [1659], "112674": 754147, "115582": 1509575, "115634": 1689223, "159501": 0, "166222": 3268045}'

In [5]:
data_train = pd.concat([df3, df4, df5, df6])

```python
""".| : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : :”-'\,,
..\: : : : : : : : : : :'\: : : : : : : : : : : : : :~,,: : : : : : : : : “~-.,_
...\ : : : : : : : : : : :\: /: : : : : : : : : : : : : : : “,: : : : : : : : : : :"~,_
... .\: : : : : : : : : : :\|: : : : : : : : :_._ : : : : : : \: : : : : : : : : : : : :”- .
... ...\: : : : : : : : : : \: : : : : : : : ( O ) : : : : : : \: : : : : : : : : : : : : : '\._
... ... .\ : : : : : : : : : '\': : : : : : : :"*": : : : : : : :|: : : : : : : : : : : : : : : |0)
... ... ...\ : : : : : : : : : '\: : : : : : : : : : : : : : : :/: : : : : : : : : : : : : : : /""
... ... .....\ : : : : : : : : : \: : : : : : : : : : : : : ,-“: : : : : : : : : : : : : : : :/
... ... ... ...\ : : : : : : : : : \: : : : : : : : : _=" : : : : : ',_.: : : : : : : :,-“
... ... ... ... \,: : : : : : : : : \: :"”'~---~”" : : : : : : : : : : : : = :"”~~"""
```

In [6]:
import torch
import torch.nn as nn
from transformers import BertModel

2025-05-05 13:56:54.999988: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746453415.296998      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746453415.378347      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class Triplet(Dataset):
    

In [ ]:
class TripletBERT(nn.Module):
    def __init__(self, text_emb_size, num_features, cat_features_size, json_emb_size):
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.num_fc = nn.Linear(num_features, 64)
        self.cat_fc = nn.Linear(cat_features_size, 64)
        self.json_fc = nn.Linear(json_emb_size, 64)
        self.fc = nn.Linear(text_emb_size + 64 * 3, 256)

    def encode(self, text_input, num_features, cat_features, json_emb):
        text_emb = self.bert(**text_input).last_hidden_state.mean(dim=1)
        num_out = torch.relu(self.num_fc(num_features))
        cat_out = torch.relu(self.cat_fc(cat_features))
        json_out = torch.relu(self.json_fc(json_emb))
        combined = torch.cat([text_emb, num_out, cat_out, json_out], dim=1)
        return torch.nn.functional.normalize(self.fc(combined), dim=1)

    def forward(self, anchor, positive, negative):
        anchor_emb = self.encode(*anchor)
        positive_emb = self.encode(*positive)
        negative_emb = self.encode(*negative)
        return anchor_emb, positive_emb, negative_emb


In [ ]:
criterion = nn.TripletMarginLoss(margin=0.5)

loss = criterion(anchor_emb, positive_emb, negative_emb)
